# Analyzing scaling with mass 
October 13, 2021 
Venkitesh Ayyar

In [137]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [138]:
%matplotlib widget

In [139]:
def f_get_site(x,y,L):
    return x+y*L

def f_get_base(xc,yc,L,quad):
    
    if quad==1: 
        xb=block_x*xc; yb=block_y*yc
    elif quad==2: 
        xb=(block_x*xc-1+L)%L; yb=block_y*yc
    elif quad==3: 
        xb=(block_x*xc-1+L)%L; yb=(block_y*yc-1+L)%L
    elif quad==4: 
        xb=block_x*xc; yb=(block_y*yc-1+L)%L        
    
    return xb,yb

def f_conv_complex(strg):
    ''' Convert input from a+ib to python complex form'''
    arr=[np.float64(k) for k in strg.split('+i')]
    return complex(arr[0],arr[1])
# f_get_base(2,2,L,quad=4)


In [143]:
# fname='../3_complex_scalar_adaptivemgrid/1_regular_tl_code/Near-null_L32_blk2_ndof2.txt'
fname='../3_complex_scalar_adaptivemgrid/2_nontl_code/Near-null_L4_blk2_ndof1.txt'
fname='../3_complex_scalar_adaptivemgrid/2_nontl_code/Near-null_L8_blk2_ndof1.txt'


# df=pd.read_csv(fname,delimiter=',',names=cols,index_col=False)
a1=np.loadtxt(fname,delimiter='\n',dtype=str)

In [144]:
arr=np.loadtxt(fname,delimiter='\n',dtype=str)
arr1=np.vectorize(f_conv_complex)(arr[:]) # Skip last element (null string) and first (iteration)

In [145]:
L=8;levels=1
block_x=2; block_y=2

In [146]:
# Build list of n_dof for each level
nc_list=[1 for i in range(levels+1)]
nc_list[0]=1

# Build list of lattice size
L_list=[(L//(block_x**i))**2 for i in range(levels)]

print(nc_list,L_list)


size_lst=[]
for i in range(levels):
    print(L_list[i],nc_list[i],L_list[i]*nc_list[i+1]*nc_list[i])
    size_lst.append(L_list[i]*nc_list[i+1]*nc_list[i])
    
print(size_lst)
print(sum(size_lst))

[1, 1] [64]
64 1 64
[64]
64


In [147]:
# 32**2 * (2*1) + 16**2 * (2*2) + 8**2 * (2*2) + 4**2 * (2*2) 

In [148]:
assert sum(size_lst)==arr1.shape[0], "Sizes unequal %d %d "%(sum(size_lst),arr1.shape[0])

In [149]:
nn_lst=[]
start_idx=0
for i in size_lst:
    nn_lst.append(arr1[start_idx:start_idx+i])
    start_idx+=i
    print(start_idx,i)

64 64


In [150]:
# # Check block normalization
# for idx in range(1,levels+1):
#     Lc=L//(block_x**idx)
#     Lf=Lc*block_x
#     print(idx,Lf,Lc)

#     for xc in range(Lc): 
#         for yc in range(Lc):
#             xb,yb=f_get_base(xc,yc,Lf,quad=1)
#             site=[f_get_site(xb+x1,yb+y1,Lf) for x1 in range(block_x) for y1 in range(block_y)]
#             # print(site)
#             norm=np.linalg.norm(np.abs(nn_lst[idx-1][site]))
#             # print(norm)
            
#             if ~np.isclose(norm,1.0,rtol=1e-20,atol=1e-14): 
#                 print("Not normalized",norm)


In [151]:
# nn_lst[-1]

In [155]:
# Check block normalization
for idx in range(0,levels):
    Lf=L//(block_x**idx)
    Lc=Lf//block_x
    print(idx,Lf,Lc)

    for xc in range(Lc): 
        for yc in range(Lc):
            xb,yb=f_get_base(xc,yc,Lf,quad=4)
            
            # print(xb,yb,end='\t')
            # print(xc*block_x,yc*block_y,end='\t')
            # site=[((xb+x1+L)%L,(yb+y1+L)%L) for x1 in range(block_x) for y1 in range(block_y)]
            # print(site)
            
            site=[f_get_site((xb+x1+Lf)%Lf,(yb+y1+Lf)%Lf,Lf) for x1 in range(block_x) for y1 in range(block_y)]

            # print(site)
            norm=np.linalg.norm(np.abs(nn_lst[idx][site]))
# #             # print(norm)
            
            if ~np.isclose(norm,1.0,rtol=1e-20,atol=1e-14): 
                print("Not normalized",norm)


0 8 4


In [134]:
for i in nn_lst:
    print(i.shape)

(64,)
